# Multi-RAG: 조건부 분기를 활용한 다중 도메인 RAG

이 노트북에서는 **조건부 분기(Conditional Edge)**를 사용하여 **질문의 도메인에 따라 다른 검색을 수행**하는 시스템을 만듭니다.

## 시나리오: 의료 상담 챗봇

```
┌────────────────────────────────────────────────────────────────────┐
│                   의료 상담 챗봇 시나리오                           │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   질문 유형 1: 의료 기록 관련                                       │
│   "제 진료 기록을 보여주세요"                                      │
│   → 의료 기록 데이터베이스에서 검색                                │
│                                                                    │
│   질문 유형 2: 보험 관련                                           │
│   "코로나 19도 보험 적용이 되나요?"                                │
│   → 보험 FAQ 데이터베이스에서 검색                                 │
│                                                                    │
│   → 질문 유형에 따라 다른 데이터 소스에서 검색해야 함!             │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 조건부 분기(Conditional Edge)란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    일반 Edge vs 조건부 Edge                         │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반 Edge:                                                       │
│   노드A ───────────────────────▶ 노드B                             │
│   (항상 같은 경로)                                                 │
│                                                                    │
│   조건부 Edge:                                                     │
│           ┌──── 조건1 ────▶ 노드B                                  │
│   노드A ──┤                                                        │
│           └──── 조건2 ────▶ 노드C                                  │
│   (조건에 따라 다른 경로)                                          │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 전체 아키텍처

```
┌────────────────────────────────────────────────────────────────────┐
│                  Multi-RAG 시스템 아키텍처                          │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   사용자 질문                                                      │
│        │                                                          │
│        ▼                                                          │
│   ┌─────────┐                                                      │
│   │ router  │  ← 도메인 분류 (records / insurance)                │
│   └────┬────┘                                                      │
│        │                                                          │
│   ┌────┴────┐  조건부 분기                                         │
│   │         │                                                      │
│   ▼         ▼                                                      │
│ ┌─────┐   ┌─────┐                                                  │
│ │의료 │   │보험 │  ← 각 도메인별 검색                              │
│ │기록 │   │FAQ  │                                                  │
│ │검색 │   │검색 │                                                  │
│ └──┬──┘   └──┬──┘                                                  │
│    │         │                                                     │
│    └────┬────┘  합류                                               │
│         │                                                          │
│         ▼                                                          │
│   ┌───────────┐                                                    │
│   │  답변생성  │  ← 검색 결과로 최종 답변                          │
│   └───────────┘                                                    │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama langgraph

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2
!ollama pull nomic-embed-text

# 2. LLM 및 임베딩 모델 설정

In [ ]:
from langchain_ollama import ChatOllama, OllamaEmbeddings

# 임베딩 모델
embeddings = OllamaEmbeddings(model='nomic-embed-text')

# 라우팅 및 정확한 답변용 (낮은 temperature)
model_low_temp = ChatOllama(model='llama3.2', temperature=0.1)

# 자연스러운 답변용 (높은 temperature)
model_high_temp = ChatOllama(model='llama3.2', temperature=0.7)

print("✅ 모델 설정 완료")

# 3. 샘플 문서 및 벡터 저장소 준비

두 가지 도메인의 문서를 준비합니다.

In [ ]:
from langchain_core.documents import Document
from langchain_core.vectorstores.in_memory import InMemoryVectorStore

# 샘플 문서
sample_docs = [
    Document(
        page_content='''# 환자 의료 기록
- 이름: 홍길동
- 성별: 남
- 나이: 30세
## 진료 내역
- 2021년 1월 13일: 감기로 인한 발열로 병원 방문
- 2022년 3월 15일: 비염 진단으로 약 처방
- 2022년 5월 20일: 피부과 진료 및 약 처방
- 2022년 6월 10일: 발열 및 기침으로 코로나 검사 및 음성 판정
- 2022년 7월 2일: 코로나 진단 검사 및 양성 판정
        ''',
        metadata={'domain': 'records'}
    ),
    Document(
        page_content='''# 보험 FAQ
- Q: 과거에 병력이 있는데 가입가능한가요?
- A: 치료기간, 현재 상태, 후유증 여부, 연령 등에 따라 다르므로 가입 가능 여부는 가까운 대리점에서 상담 받으세요.

- Q: 보험금 청구 방법은 어떻게 되나요??
- A: 보험금 청구는 보험금 청구서 작성 후 가까운 대리점을 통해 진행 가능합니다.

- Q: 코로나 19도 보험 적용이 되나요?
- A: 코로나 19는 보험 대상에 포함되지 않습니다.

- Q: 보험금 지급이 거부되는 경우는 어떤 경우인가요?
- A: 보험금 지급 거부 사유는 보험 계약서에 명시되어 있습니다. 자세한 사항은 계약서를 참조하세요
        ''',
        metadata={'domain': 'insurance'}
    ),
]

print("=== 샘플 문서 ===")
for doc in sample_docs:
    print(f"\n도메인: {doc.metadata['domain']}")
    print(f"내용: {doc.page_content[:100]}...")

In [ ]:
# 벡터 저장소 생성 (도메인별)
medical_records_store = InMemoryVectorStore.from_documents(sample_docs, embeddings)
medical_records_retriever = medical_records_store.as_retriever()

insurance_faqs_store = InMemoryVectorStore.from_documents(sample_docs, embeddings)
insurance_faqs_retriever = insurance_faqs_store.as_retriever()

print("✅ 벡터 저장소 준비 완료")
print("   - medical_records_retriever: 의료 기록 검색")
print("   - insurance_faqs_retriever: 보험 FAQ 검색")

# 4. State 정의

In [ ]:
from typing import Annotated, Literal, TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    """
    Multi-RAG 시스템의 상태
    """
    messages: Annotated[list, add_messages]    # 대화 기록
    user_query: str                            # 사용자 질문
    domain: Literal['records', 'insurance']    # 분류된 도메인
    documents: list[Document]                  # 검색된 문서
    answer: str                                # 최종 답변

class Input(TypedDict):
    user_query: str

class Output(TypedDict):
    documents: list[Document]
    answer: str

print("✅ State 정의 완료")

# 5. 노드 정의

## 노드 1: Router (도메인 분류)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

# 라우터 프롬프트
router_prompt = SystemMessage(
    content='''사용자 문의를 어느 도메인으로 라우팅할지 결정하세요. 선택할 수 있는 두 가지 도메인은 다음과 같습니다.
- records: 진단, 치료, 처방과 같은 환자의 의료 기록을 포함합니다.  
- insurance: 보험 정책, 청구, 보장에 대한 자주 묻는 질문을 포함합니다.  

도메인 이름만 출력하세요.'''
)

def router_node(state: State) -> State:
    """
    Router 노드: 질문의 도메인을 분류
    
    출력: 'records' 또는 'insurance'
    """
    user_message = HumanMessage(content=state['user_query'])
    messages = [router_prompt, *state['messages'], user_message]
    
    res = model_low_temp.invoke(messages)
    
    # 도메인 결정 (출력에서 도메인 추출)
    domain = 'records' if 'records' in res.content.lower() else 'insurance'
    
    print(f"🔀 Router: '{state['user_query'][:30]}...' → {domain}")
    
    return {
        'domain': domain,
        'messages': [user_message, res],
    }

print("✅ router_node 정의 완료")

## 조건부 분기 함수

In [ ]:
def pick_retriever(state: State) -> Literal['retrieve_medical_records', 'retrieve_insurance_faqs']:
    """
    조건부 분기 함수
    
    도메인에 따라 다음 노드를 결정:
    - records → retrieve_medical_records
    - insurance → retrieve_insurance_faqs
    """
    if state['domain'] == 'records':
        return 'retrieve_medical_records'
    else:
        return 'retrieve_insurance_faqs'

print("✅ pick_retriever 조건부 분기 함수 정의 완료")

## 노드 2a, 2b: 검색 노드들

In [ ]:
def retrieve_medical_records(state: State) -> State:
    """
    의료 기록 검색 노드
    """
    documents = medical_records_retriever.invoke(state['user_query'])
    print(f"🏥 의료 기록 검색: {len(documents)}개 문서 찾음")
    return {'documents': documents}

def retrieve_insurance_faqs(state: State) -> State:
    """
    보험 FAQ 검색 노드
    """
    documents = insurance_faqs_retriever.invoke(state['user_query'])
    print(f"📋 보험 FAQ 검색: {len(documents)}개 문서 찾음")
    return {'documents': documents}

print("✅ 검색 노드들 정의 완료")

## 노드 3: 답변 생성

In [ ]:
# 도메인별 프롬프트
medical_records_prompt = SystemMessage(
    content='당신은 유능한 의료 챗봇입니다. 진단, 치료, 처방과 같은 환자의 의료 기록을 기반으로 질문에 답하세요.'
)

insurance_faqs_prompt = SystemMessage(
    content='당신은 유능한 의료 보험 챗봇입니다. 보험 정책, 청구 및 보장에 대한 자주 묻는 질문에 답하세요.'
)

def generate_answer(state: State) -> State:
    """
    답변 생성 노드
    
    도메인에 맞는 프롬프트를 사용하여 답변 생성
    """
    # 도메인에 따른 프롬프트 선택
    if state['domain'] == 'records':
        prompt = medical_records_prompt
    else:
        prompt = insurance_faqs_prompt
    
    messages = [
        prompt,
        *state['messages'],
        HumanMessage(content=f'Documents: {state["documents"]}'),
    ]
    
    res = model_high_temp.invoke(messages)
    
    return {
        'answer': res.content,
        'messages': res,
    }

print("✅ generate_answer 노드 정의 완료")

# 6. 그래프 구성 (조건부 분기 포함)

In [ ]:
from langgraph.graph import StateGraph, START, END

# 그래프 빌더
builder = StateGraph(State, input=Input, output=Output)

# 노드 추가
builder.add_node('router', router_node)
builder.add_node('retrieve_medical_records', retrieve_medical_records)
builder.add_node('retrieve_insurance_faqs', retrieve_insurance_faqs)
builder.add_node('generate_answer', generate_answer)

# 엣지 추가
builder.add_edge(START, 'router')

# ⭐ 조건부 엣지 추가
builder.add_conditional_edges('router', pick_retriever)

# 검색 노드들 → 답변 생성
builder.add_edge('retrieve_medical_records', 'generate_answer')
builder.add_edge('retrieve_insurance_faqs', 'generate_answer')

# 답변 생성 → 종료
builder.add_edge('generate_answer', END)

# 그래프 컴파일
graph = builder.compile()

print("✅ 그래프 컴파일 완료 (조건부 분기 포함)")

In [ ]:
# 그래프 구조 시각화
print("=== 그래프 구조 ===")
print(graph.get_graph().draw_mermaid())

# 7. Multi-RAG 실행 테스트

In [ ]:
# 보험 관련 질문 테스트
print("=== 보험 관련 질문 ===")
input_data = {'user_query': '코로나 19도 보험 적용이 되나요?'}

print(f"\n질문: {input_data['user_query']}\n")

for chunk in graph.stream(input_data):
    for key, value in chunk.items():
        if key == 'generate_answer':
            print(f"\n💬 답변:\n{value['answer']}")

In [ ]:
# 의료 기록 관련 질문 테스트
print("\n=== 의료 기록 관련 질문 ===")
input_data = {'user_query': '홍길동 환자의 코로나 진단 기록을 알려주세요.'}

print(f"\n질문: {input_data['user_query']}\n")

for chunk in graph.stream(input_data):
    for key, value in chunk.items():
        if key == 'generate_answer':
            print(f"\n💬 답변:\n{value['answer']}")

# 8. 다양한 질문으로 테스트

In [ ]:
test_queries = [
    "보험금 청구는 어떻게 하나요?",
    "환자의 최근 진료 내역은?",
    "보험금 지급 거부 사유가 뭔가요?",
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"📝 질문: {query}")
    print("="*60)
    
    result = graph.invoke({'user_query': query})
    
    print(f"\n💬 답변:\n{result['answer'][:300]}...")

---

## 정리: Multi-RAG 시스템

### 조건부 분기 핵심 코드

```python
# 1. 조건부 분기 함수 정의
def pick_retriever(state):
    if state['domain'] == 'records':
        return 'retrieve_medical_records'
    else:
        return 'retrieve_insurance_faqs'

# 2. 조건부 엣지 추가
builder.add_conditional_edges('router', pick_retriever)
```

### 아키텍처 패턴

```
┌─────────────────────────────────────────────────────────────────────┐
│                Multi-RAG 패턴                                       │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   1. Router 노드: 질문 분류                                         │
│   2. 조건부 분기: 도메인별 검색 노드 선택                            │
│   3. 검색 노드: 각 도메인 데이터 검색                               │
│   4. 답변 생성: 검색 결과로 최종 답변                               │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 일반 Edge vs 조건부 Edge

| 유형 | 사용법 | 용도 |
|------|--------|------|
| **add_edge** | `builder.add_edge(A, B)` | 항상 A → B |
| **add_conditional_edges** | `builder.add_conditional_edges(A, func)` | 조건에 따라 분기 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
model = ChatOpenAI(model='gpt-4o-mini')

# 변경
from langchain_ollama import ChatOllama, OllamaEmbeddings
embeddings = OllamaEmbeddings(model='nomic-embed-text')
model = ChatOllama(model='llama3.2')
```

## ch05 요약

| 노트북 | 주제 | 핵심 기술 |
|--------|------|----------|
| **01-03** | 기본 챗봇 | State, Node, Edge |
| **04-05** | SQL 생성기 | 다중 노드 순차 연결 |
| **06-07** | Multi-RAG | 조건부 분기 (conditional_edges) |